In [1]:
import pandas as pd
from pandas import ExcelWriter
from openpyxl import Workbook
from openpyxl.styles import Alignment

In [2]:
import sqlite3
conn = sqlite3.connect('nepali_sabdakosh.sqlite3')

In [19]:
JOIN_QUERY = '''
SELECT w.id as `word_id`, w.value AS `word`, w.part_of_speech, 
d.id as `definition_id`, d.value AS `definition`, 
e.id as `example_id`, e.value as `example`
	FROM `word` w 
	LEFT JOIN `definition` d ON w.id = d.word_id
    LEFT JOIN `example` e ON d.id = e.definition_id;'''
df = pd.read_sql_query(JOIN_QUERY, conn)

In [20]:
df.head()

,word_id,word,part_of_speech,definition_id,definition,example_id,example
0,1,अँ,नि.,1.0,कुनै कुरा मान्छु वा हो भन्ने बुझाउन प्रयोग गरि...,1.0,अँ बुझेँ — बोलीको साथसाथै ऊ सिरकलाई फेरि तानेर...
1,1,अँ,नि.,2.0,कुनै कुरा मान्दिन वा गर्दिन भन्ने बुझाउन प्रयो...,2.0,अँ छोडुँला!
2,1,अँ,नि.,2.0,कुनै कुरा मान्दिन वा गर्दिन भन्ने बुझाउन प्रयो...,3.0,अँ हुन्थ्यो!
3,1,अँ,नि.,3.0,कुनै नयाँ प्रसङ्ग सुरु गर्न वा केही बेर रोकिएर...,4.0,अँ यो कहाँ रहिछ?
4,1,अँ,नि.,3.0,कुनै नयाँ प्रसङ्ग सुरु गर्न वा केही बेर रोकिएर...,5.0,"अँ त म भन्दै थिएँ, यो बाटोमा भेटियो।"


In [23]:
df = df.drop(columns={''}, errors='ignore')
df.to_csv('nepali_sabdakosh.csv', index=False)

,word_id,word,part_of_speech,definition_id,definition,example_id,example
0,1,अँ,नि.,1.0,कुनै कुरा मान्छु वा हो भन्ने बुझाउन प्रयोग गरि...,1.0,अँ बुझेँ — बोलीको साथसाथै ऊ सिरकलाई फेरि तानेर...
1,1,अँ,नि.,2.0,कुनै कुरा मान्दिन वा गर्दिन भन्ने बुझाउन प्रयो...,2.0,अँ छोडुँला!
2,1,अँ,नि.,2.0,कुनै कुरा मान्दिन वा गर्दिन भन्ने बुझाउन प्रयो...,3.0,अँ हुन्थ्यो!
3,1,अँ,नि.,3.0,कुनै नयाँ प्रसङ्ग सुरु गर्न वा केही बेर रोकिएर...,4.0,अँ यो कहाँ रहिछ?
4,1,अँ,नि.,3.0,कुनै नयाँ प्रसङ्ग सुरु गर्न वा केही बेर रोकिएर...,5.0,"अँ त म भन्दै थिएँ, यो बाटोमा भेटियो।"


In [3]:
FETCH_WORD_QUERY = "SELECT * FROM `word`"

word_df = pd.read_sql_query(FETCH_WORD_QUERY, conn)
definition_df = pd.read_sql_query("SELECT * FROM `definition`", conn)
example_df = pd.read_sql_query("SELECT * FROM `example`", conn)

In [4]:
# stat = {"word_df": word_df.isnull().sum(),
#         "definition_df": definition_df.isnull().sum(),
#         "example_df": example_df.isnull().sum()}
# print(stat)

In [5]:
word_df.rename(columns={'id': 'word_id'}, inplace=True)
word_df.head()

,word_id,value,part_of_speech
0,1,अँ,नि.
1,2,अँगाल्नु,एक. क्रि.
2,3,अँग्रेजी,N/A
3,4,अँजुली,सङ्. ना.
4,5,अँधेरो,N/A


In [6]:
definition_df.rename(columns={'id': 'definition_id'}, inplace = True)
definition_df.head()

,definition_id,word_id,value
0,1,1,कुनै कुरा मान्छु वा हो भन्ने बुझाउन प्रयोग गरि...
1,2,1,कुनै कुरा मान्दिन वा गर्दिन भन्ने बुझाउन प्रयो...
2,3,1,कुनै नयाँ प्रसङ्ग सुरु गर्न वा केही बेर रोकिएर...
3,4,1,कुरा गर्दागर्दै अलमलिँदा वा बिर्सँदा खाली ठाउँ...
4,5,2,एक वा दुवै हातले अर्काको गर्धन बेर्नु


In [7]:
example_df.rename(columns={'id': 'example_id'}, inplace = True)
example_df.head()

,example_id,definition_id,value
0,1,1,अँ बुझेँ — बोलीको साथसाथै ऊ सिरकलाई फेरि तानेर...
1,2,2,अँ छोडुँला!
2,3,2,अँ हुन्थ्यो!
3,4,3,अँ यो कहाँ रहिछ?
4,5,3,"अँ त म भन्दै थिएँ, यो बाटोमा भेटियो।"


In [8]:
definition_with_eg = pd.merge(definition_df, example_df, on=["definition_id"])
print(definition_with_eg.count())
definition_with_eg.head()

definition_id    16115
word_id          16115
value_x          16115
example_id       16115
value_y          16115
dtype: int64


,definition_id,word_id,value_x,example_id,value_y
0,1,1,कुनै कुरा मान्छु वा हो भन्ने बुझाउन प्रयोग गरि...,1,अँ बुझेँ — बोलीको साथसाथै ऊ सिरकलाई फेरि तानेर...
1,2,1,कुनै कुरा मान्दिन वा गर्दिन भन्ने बुझाउन प्रयो...,2,अँ छोडुँला!
2,2,1,कुनै कुरा मान्दिन वा गर्दिन भन्ने बुझाउन प्रयो...,3,अँ हुन्थ्यो!
3,3,1,कुनै नयाँ प्रसङ्ग सुरु गर्न वा केही बेर रोकिएर...,4,अँ यो कहाँ रहिछ?
4,3,1,कुनै नयाँ प्रसङ्ग सुरु गर्न वा केही बेर रोकिएर...,5,"अँ त म भन्दै थिएँ, यो बाटोमा भेटियो।"


In [9]:
words = pd.merge(word_df, definition_with_eg, on=["word_id"])
print(words.count())
words.head()

word_id           16115
value             16115
part_of_speech    16115
definition_id     16115
value_x           16115
example_id        16115
value_y           16115
dtype: int64


,word_id,value,part_of_speech,definition_id,value_x,example_id,value_y
0,1,अँ,नि.,1,कुनै कुरा मान्छु वा हो भन्ने बुझाउन प्रयोग गरि...,1,अँ बुझेँ — बोलीको साथसाथै ऊ सिरकलाई फेरि तानेर...
1,1,अँ,नि.,2,कुनै कुरा मान्दिन वा गर्दिन भन्ने बुझाउन प्रयो...,2,अँ छोडुँला!
2,1,अँ,नि.,2,कुनै कुरा मान्दिन वा गर्दिन भन्ने बुझाउन प्रयो...,3,अँ हुन्थ्यो!
3,1,अँ,नि.,3,कुनै नयाँ प्रसङ्ग सुरु गर्न वा केही बेर रोकिएर...,4,अँ यो कहाँ रहिछ?
4,1,अँ,नि.,3,कुनै नयाँ प्रसङ्ग सुरु गर्न वा केही बेर रोकिएर...,5,"अँ त म भन्दै थिएँ, यो बाटोमा भेटियो।"


In [10]:
words.rename(columns={'value': 'word', 'value_x': 'definition', 'value_y': 'example'}, inplace = True)
# words = words.drop(columns={'id'}, errors='ignore')
words

,word_id,word,part_of_speech,definition_id,definition,example_id,example
0,1,अँ,नि.,1,कुनै कुरा मान्छु वा हो भन्ने बुझाउन प्रयोग गरि...,1,अँ बुझेँ — बोलीको साथसाथै ऊ सिरकलाई फेरि तानेर...
1,1,अँ,नि.,2,कुनै कुरा मान्दिन वा गर्दिन भन्ने बुझाउन प्रयो...,2,अँ छोडुँला!
2,1,अँ,नि.,2,कुनै कुरा मान्दिन वा गर्दिन भन्ने बुझाउन प्रयो...,3,अँ हुन्थ्यो!
3,1,अँ,नि.,3,कुनै नयाँ प्रसङ्ग सुरु गर्न वा केही बेर रोकिएर...,4,अँ यो कहाँ रहिछ?
4,1,अँ,नि.,3,कुनै नयाँ प्रसङ्ग सुरु गर्न वा केही बेर रोकिएर...,5,"अँ त म भन्दै थिएँ, यो बाटोमा भेटियो।"
...,...,...,...,...,...,...,...
16110,7606,कर लाग्नु,क्रि.,10029,कुनै काम गर्न वा कुनै कुरा मान्न बाध्य हुनु,16111,कतिपय परिस्थितिजन्य आधारहरूले पनि उनको गिरफ्त...
16111,7607,बक फुट्नु,क्रि.,10030,बोली निस्कनु,16112,एकछिन त स्नेह विह्वल भए र बक नै फुटेन।
16112,7608,साक्षी बक्नु,क्रि.,10031,कुनै घटना वा व्यक्तिका बारेमा आफुले यो कुरा जा...,16113,बात लागेका व्यक्तिको पक्षमा मन्त्री हुँदा साक...
16113,7609,कसार बटार्नु,क्रि.,10032,कसार बनाउन पिठो डल्लो पार्नु,16114,"वारि घर पारि घर कसार बटाराइ छ, खानु पिउनु केह..."


In [ ]:
words.to_csv('nepali_sabdakosh.csv')

In [20]:
from openpyxl.utils.dataframe import dataframe_to_rows

workbook = Workbook()
ws = workbook.active

for r in dataframe_to_rows(words, index=True, header=True):
    ws.append(r)

key_column = 1
merge_columns = [2, 3, 4, 5, 6, 7, 8]
start_row = 1
max_row = ws.max_row
key = None


# workbook.save('dict.xlsx')

In [ ]:
# dataframe = pd.DataFrame(words).set_index(["word_id", "word"])   
# with ExcelWriter("nepali_sabdakosh.xlsx") as writer:
#     dataframe.to_excel(writer)

In [14]:
# writer = pd.ExcelWriter('nepali_sabdakosh.xlsx', engine='xlsxwriter')
# words.to_excel(writer, sheet_name = 'Overview') 
# workbook  = writer.book
# worksheet_O = writer.sheets['Overview']
# header_format = workbook.add_format({'bold': True, 'text_wrap': False, 'valign': 'top', 'border': 1})

# for col_num, value in enumerate(dataframe.columns.values):
#     worksheet_O.write(0, col_num + 1, value, header_format)

# writer.save()